In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import lognorm


# generate samples

In [106]:
def generate_log_normal_with_noise(coefficients, num_samples=10000):
    np.random.seed(42)
     
    # Save this , as this is the core relationship. If there is treatment, we can assume it is this coefficient changed
    # For any treatment, some of this would be positive, some of this could be negative. 
    features = np.random.rand(num_samples, 10) # User feature
    # Compute the mean as a linear combination of the features
    mean = np.dot(features, coefficients)
    y = np.random.lognormal(mean=np.log(mean), sigma=1.5)
    print(y.mean(), y.std())
    # Generate 30 random features as noise
    noise = np.random.rand(num_samples, 30)

    # Create a DataFrame
    column_names = [f'feature_{i+1}' for i in range(10)] + [f'noise_{i+1}' for i in range(30)] + ['y']
    data = np.hstack((features, noise, y.reshape(-1, 1)))  # Combine all data
    df = pd.DataFrame(data, columns=column_names)

    return df

# Example usage
np.random.seed(42)
COEFFICIENT = np.random.rand(10)+1
TREATMENT_EFFECT_ON_COEF = 0.1
df_control = generate_log_normal_with_noise(COEFFICIENT)
df_treatment = generate_log_normal_with_noise(COEFFICIENT+TREATMENT_EFFECT_ON_COEF)
df_control['t'] = 0
df_treatment['t'] = 1
df_raw = pd.concat([df_control,df_treatment])
#print(df.head())

23.713607071959036 64.57858212394598
25.27453304559599 68.83774314801202


In [107]:
df_raw.groupby('t')['y'].mean()

t
0    23.713607
1    25.274533
Name: y, dtype: float64

In [108]:
24.992431/23.449822,24.992431 - 23.449822

(1.065783399123456, 1.5426089999999988)

# T-test vs. bootstrap

In [112]:
import pandas as pd
from scipy import stats

# Sample DataFrame
def t_test(df):
    # Separate the groups
    control_group = df[df['t'] == 0]['y']
    treatment_group = df[df['t'] == 1]['y']
    
        # Calculate means
    mean_control = control_group.mean()
    mean_treatment = treatment_group.mean()
    
    # Calculate the mean difference
    mean_diff = mean_treatment - mean_control
    
    # Calculate standard errors
    se_control = control_group.std(ddof=1) / (len(control_group) ** 0.5)  # Standard error for control
    se_treatment = treatment_group.std(ddof=1) / (len(treatment_group) ** 0.5)  # Standard error for treatment
    
    # Calculate the standard error of the mean difference
    se_diff = (se_control**2 + se_treatment**2) ** 0.5
    
    # Calculate the degrees of freedom
    df = len(control_group) + len(treatment_group) - 2
    
    # Determine the critical t-value for 95% confidence interval
    alpha = 0.05
    t_critical = stats.t.ppf(1 - alpha/2, df)
    
    # Compute the confidence interval for the mean difference
    ci_lower = mean_diff - t_critical * se_diff
    ci_upper = mean_diff + t_critical * se_diff
    
    # Perform the t-test
    t_stat, p_value = stats.ttest_ind(control_group, treatment_group)
    
    # Output the results
    print(f'Mean Difference: {mean_diff}')
    print(f'95% Confidence Interval for Mean Difference: ({ci_lower}, {ci_upper})')
    print(f'T-statistic: {t_stat}, P-value: {p_value}')
    
    # Interpret the results
    if p_value < alpha:
        print("Reject the null hypothesis: There is a significant difference between the two groups.")
    else:
        print("Fail to reject the null hypothesis: No significant difference between the two groups.")

In [113]:
t_test(df_raw[['t','y']])

Mean Difference: 1.5609259736369552
95% Confidence Interval for Mean Difference: (-0.2892424239617504, 3.4110943712356607)
T-statistic: -1.653656970551974, P-value: 0.09821292554761378
Fail to reject the null hypothesis: No significant difference between the two groups.


In [114]:
def stratified_bootstrap(data, n_iterations=1000):
    means = []
    for _ in range(n_iterations):
        # Sample within each stratum
        control_sample = data[data['t'] == 0]['y'].sample(frac=1, replace=True)
        treatment_sample = data[data['t'] == 1]['y'].sample(frac=1, replace=True)
        # Calculate mean difference
        mean_diff = treatment_sample.mean() - control_sample.mean()
        means.append(mean_diff)
    return np.array(means)

# Perform stratified bootstrap
bootstrap_means = stratified_bootstrap(df_raw)

# Calculate confidence intervals
ci_lower = np.percentile(bootstrap_means, 2.5)
ci_upper = np.percentile(bootstrap_means, 97.5)

print(f'95% Confidence Interval for Mean Difference: ({ci_lower}, {ci_upper})')

95% Confidence Interval for Mean Difference: (-0.2143743617836951, 3.3430522214711047)


In [117]:
def stratified_bootstrap(data, features, n_iterations=1000):
    means = []
    
    # Create quantiles for each feature and store the quantile columns
    quantile_columns = []
    for feature in features:
        quantile_col_name = f"{feature}_quantile"
        data[quantile_col_name] = pd.qcut(data[feature], q=4, labels=False)  # Create quartiles
        quantile_columns.append(quantile_col_name)

    for _ in range(n_iterations):
        # Sample within each quantile stratum
        strata = data.groupby(['t'] + quantile_columns)
        control_samples = []
        treatment_samples = []
        
        for name, group in strata:
            control_samples.append(group[group['t'] == 0]['y'].sample(frac=1, replace=True))
            treatment_samples.append(group[group['t'] == 1]['y'].sample(frac=1, replace=True))
        
        # Concatenate all samples from each stratum
        control_sample = pd.concat(control_samples)
        treatment_sample = pd.concat(treatment_samples)
        
        # Calculate mean difference
        mean_diff = treatment_sample.mean() - control_sample.mean()
        means.append(mean_diff)

    return np.array(means)

# Specify features to include in the stratification
features = ['feature_1', 'feature_2', 'feature_3']  # Add features up to feature_10 as needed

# Perform stratified bootstrap
bootstrap_means = stratified_bootstrap(df_raw, features)

# Calculate confidence intervals
ci_lower = np.percentile(bootstrap_means, 2.5)
ci_upper = np.percentile(bootstrap_means, 97.5)

print(f'95% Confidence Interval for Mean Difference: ({ci_lower}, {ci_upper})')

95% Confidence Interval for Mean Difference: (-0.3638831290383325, 3.482796708909251)


In [119]:
# faster version?
def stratified_bootstrap(data, features, n_iterations=1000):
    means = np.zeros(n_iterations)  # Preallocate space for means
    
    # Create quantiles for each feature and store the quantile columns
    for feature in features:
        quantile_col_name = f"{feature}_quantile"
        data[quantile_col_name] = pd.qcut(data[feature], q=4, labels=False)

    # Group data by treatment and quantile columns
    strata = data.groupby(['t'] + [f"{feature}_quantile" for feature in features])
    
    for i in range(n_iterations):
        treatment_samples = []
        control_samples = []
        
        # Sample within each stratum
        for name, group in strata:
            sample_size = len(group)
            if group['t'].iloc[0] == 1:  # Treatment group
                treatment_samples.append(group['y'].sample(sample_size, replace=True))
            else:  # Control group
                control_samples.append(group['y'].sample(sample_size, replace=True))
        
        # Concatenate samples and calculate mean difference
        if treatment_samples and control_samples:
            treatment_sample = np.concatenate(treatment_samples)
            control_sample = np.concatenate(control_samples)
            means[i] = treatment_sample.mean() - control_sample.mean()

    return means

# Specify features to include in the stratification
features = ['feature_1', 'feature_2', 'feature_3']  # Add features up to feature_10 as needed

# Perform stratified bootstrap
bootstrap_means = stratified_bootstrap(df_raw, features)

# Calculate confidence intervals
ci_lower = np.percentile(bootstrap_means, 2.5)
ci_upper = np.percentile(bootstrap_means, 97.5)

print(f'95% Confidence Interval for Mean Difference: ({ci_lower}, {ci_upper})')

95% Confidence Interval for Mean Difference: (-0.24158057057094603, 3.371164904970394)


In [ ]:
# use other method